# Bayesian Spatio-Temporal Graph Transformer Network (B-STAR) for Multi-Aircraft Trajectory Prediction
Author: Yutian Pang, Arizona State University


Email: yutian.pang@asu.edu

# Part 1: IFF ASDE-X Flight Track Data Processing with PySpark and Hadoop
This is a demonstration of the process of using PySpark and Hadoop for large-scale processing of IFF ASDE-X data. In practice, this data processing would be performed on a server via ssh.

## Module Requirements

This Jupyter notebook has been tested with:
- Ubuntu 20.04 LTS (and 18.04 LTS)
- Python 3.8.5 (and 3.8.10)
- Spark 3.1.1 with Hadoop3.2 (and Spark 3.2.1 with Hadoop3.2)

The software in parenthesis were tested together. Other combinations of Ubuntu, Python, and Spark should be verified for compatibility. See [this](https://stackoverflow.com/questions/58700384/how-to-fix-typeerror-an-integer-is-required-got-type-bytes-error-when-tryin) article for further guidance.

### Windows 10 Users
#### Installing Ubuntu on Windows 10 with Windows Subsystem for LInux (WSL)
Windows 10 users with admin privileges can enable Windows Subsystem for Linux (WSL) following [these](https://docs.microsoft.com/en-us/windows/wsl/install-win10) directions. 


#### Installing Anaconda on WSL
A user can then install Anaconda on their WSL Ubuntu distribution following [these](https://gist.github.com/kauffmanes/5e74916617f9993bc3479f401dfec7da) instructions.

#### Download and unzipping Spark on WSL
Identify the distribution of Spark and Hadoop you require [here](https://spark.apache.org/downloads.html). In your Ubuntu terminal window execute the ```wget``` command followed by the download link in your chosen download directory (likely the ```HOME``` directory). Then, unzip the downloaded .tgz file with ```tar -xvzf [fname]```.



## Installing the required Python packages
The required Python packages for this module are:
- [```pyspark```](http://spark.apache.org/docs/latest/api/python/getting_started/index.html)
    - This is the Python API for Apache Spark. We will be using the distributed processing features (with the help of the partitioned ***resilient distributed dataset*** (RDD)) and backend SQL queries for structured data.
- [```sedona```](https://sedona.apache.org/)
    - This is the geo-spatial extension of PySpark with ***Spatial RDDs*** and Spatial SQL for large-scale distributed geo-spatial data processing.
    
In the Ubuntu or Anaconda terminal, execute ```pip install pyspark apache-sedona```. This will install both the ```pyspark``` and ```sedona``` packages. 

See more on the installati```sedona```

## Setting Environment Variables
The Spark codes (note: Improve this description) retrieve the ```SPARK_HOME```, ```PYTHONPATH```, ```PYSPARK_PYTHON```, and ```PYSPARK_DRIVER_PYTHON``` system variables. Either (Option 1) these are set in the shell environment in the ```.bash_profile``` script or (Option 2) in the Python script prior to calling the ```pyspark``` module.
### Option 1: Add environment variables to the ```.bash_profile``` script
Open the ```.bash_profile``` script in your text editor. On Ubuntu systems, this script is usually found in your ```HOME``` directory ```~/```. If this file does not yet exist (or is empty) you can create one. Then add the following ```export``` statements for each variable you want to add and add them to the path. For example:

```export SPARK_HOME="$HOME/spark-X.X.X-bin-hadoopX.X"```

```export PYTHONPATH="$HOME/anacond3/bin/python3.8"```

```export PYSPARK_PYTHON="$HOME/anacond3/bin/python3.8"```

```export PYSPARK_DRIVER_PYTHON="$HOME/anacond3/bin/python3.8"```

```export PATH="$SPARK_HOME/bin:$PATH"```


### Option 2: Add the environment variables in the Python script using the ```os``` package
```import os```

```os.environ["SPARK_HOME"] = '/home/ypang6/spark-3.1.1-bin-hadoop3.2'```

```os.environ["PYTHONPATH"] = '/home/ypang6/anaconda3/bin/python3.8'```

```os.environ['PYSPARK_PYTHON'] = '/home/ypang6/anaconda3/bin/python3.8'```

```os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ypang6/anaconda3/bin/python3.8'```


## Define SQL Schema with pyspark

In [1]:
## Import Spark Session module
# We need SQL utilities from Spark
from pyspark.sql import SparkSession
from pyspark.sql.types import (ShortType, StringType, StructType,StructField,LongType, IntegerType, DoubleType)

def load_schema():
    myschema = StructType([
        StructField("recType", ShortType(), True),  # 1  //track point record type number
        StructField("recTime", StringType(), True),  # 2  //seconds since midnigght 1/1/70 UTC
        StructField("fltKey", LongType(), True),  # 3  //flight key
        StructField("bcnCode", IntegerType(), True),  # 4  //digit range from 0 to 7
        StructField("cid", IntegerType(), True),  # 5  //computer flight id
        StructField("Source", StringType(), True),  # 6  //source of the record
        StructField("msgType", StringType(), True),  # 7
        StructField("acId", StringType(), True),  # 8  //call sign
        StructField("recTypeCat", IntegerType(), True),  # 9
        StructField("lat", DoubleType(), True),  # 10
        StructField("lon", DoubleType(), True),  # 11
        StructField("alt", DoubleType(), True),  # 12  //in 100s of feet
        StructField("significance", ShortType(), True),  # 13 //digit range from 1 to 10
        StructField("latAcc", DoubleType(), True),  # 14
        StructField("lonAcc", DoubleType(), True),  # 15
        StructField("altAcc", DoubleType(), True),  # 16
        StructField("groundSpeed", IntegerType(), True),  # 17 //in knots
        StructField("course", DoubleType(), True),  # 18  //in degrees from true north
        StructField("rateOfClimb", DoubleType(), True),  # 19  //in feet per minute
        StructField("altQualifier", StringType(), True),  # 20  //Altitude qualifier (the “B4 character”)
        StructField("altIndicator", StringType(), True),  # 21  //Altitude indicator (the “C4 character”)
        StructField("trackPtStatus", StringType(), True),  # 22  //Track point status (e.g., ‘C’ for coast)
        StructField("leaderDir", IntegerType(), True),  # 23  //int 0-8 representing the direction of the leader line
        StructField("scratchPad", StringType(), True),  # 24
        StructField("msawInhibitInd", ShortType(), True),  # 25 // MSAW Inhibit Indicator (0=not inhibited, 1=inhibited)
        StructField("assignedAltString", StringType(), True),  # 26
        StructField("controllingFac", StringType(), True),  # 27
        StructField("controllingSec", StringType(), True),  # 28
        StructField("receivingFac", StringType(), True),  # 29
        StructField("receivingSec", StringType(), True),  # 30
        StructField("activeContr", IntegerType(), True),  # 31  // the active control number
        StructField("primaryContr", IntegerType(), True),
        # 32  //The primary(previous, controlling, or possible next)controller number
        StructField("kybrdSubset", StringType(), True),  # 33  //identifies a subset of controller keyboards
        StructField("kybrdSymbol", StringType(), True),  # 34  //identifies a keyboard within the keyboard subsets
        StructField("adsCode", IntegerType(), True),  # 35  //arrival departure status code
        StructField("opsType", StringType(), True),  # 36  //Operations type (O/E/A/D/I/U)from ARTS and ARTS 3A data
        StructField("airportCode", StringType(), True),  # 37
        StructField("trackNumber", IntegerType(), True),  # 38
        StructField("tptReturnType", StringType(), True),  # 39
        StructField("modeSCode", StringType(), True)  # 40
    ])
    return myschema

## Use Sedona to register Spark Objects

In [2]:
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

spark = SparkSession.\
        builder.\
        master("local[*]").\
        appName("Sector_IFF_Parser").\
        config("spark.serializer", KryoSerializer.getName).\
        config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
        config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.datasyslab:geotools-wrapper:geotools-24.0") .\
        getOrCreate()

SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext

# Load Data
## We use ASDE-X flight recordings here 

In [3]:
import glob
iff_schema = load_schema()
# The date of ASDE-X we are going to process
date = 20190807

# The path to the csv file
data_path = "/PATH_TO_CSV/IFF_ATL+ASDEX_{}*.csv".format(date)
# For example,
data_path = "/media/ypang6/paralab/Research/data/ATL/IFF_ATL+ASDEX_{}*.csv".format(date)

In [4]:
def load_data(date):
    file_path = glob.glob(data_path)[0]
    df = spark.read.csv(file_path, header=False, sep=",", schema=iff_schema)
    print("Date: {} Number of Lines: {}".format(date, df.count()))
    return df

In [5]:
# load data into Sedona
df = load_data(date)

Date: 20190807 Number of Lines: 1699663


In [6]:
# In this work, we are only interested in the flight ID, timestamps, and coordinates(latitude, longitude, altitude)
cols = ['recType', 'recTime', 'acId', 'lat', 'lon', 'alt']
df = df.select(*cols).filter(df['recType']==3).withColumn("recTime", df['recTime'].cast(IntegerType()))
df.show(5)

+-------+----------+----+--------+---------+-----+
|recType|   recTime|acId|     lat|      lon|  alt|
+-------+----------+----+--------+---------+-----+
|      3|1565149748|UNKN|33.63186|-84.44462|10.06|
|      3|1565149749|UNKN|33.63186|-84.44455|10.06|
|      3|1565149752|UNKN|33.63183|-84.44448|10.06|
|      3|1565149753|UNKN|33.63183|-84.44443|10.06|
|      3|1565149754|UNKN|33.63183|-84.44438|10.06|
+-------+----------+----+--------+---------+-----+
only showing top 5 rows



In [7]:
# Maximum altitude in the data is 8,500 feet
df.agg({"alt": "max"}).collect()[0][0]

85.0

### Build Spatial Dataframe with Sedona

In [8]:
# time window to query
duration = 4 # hours # para-atm time window filtering function
t_start = 1564668000 + (date-20190801)*24*3600 # Aug 1st, 2pm, 2019, UTC
t_end = t_start + 3600*duration

In [9]:
# register pyspark df in SQL
df.registerTempTable("pointtable")  # now we have a registered SQL table running backened in the system

# create shape column in geospark
spatialdf = spark.sql(
  """
  SELECT ST_Point(CAST(lat AS Decimal(24, 20)), CAST(lon AS Decimal(24, 20))) AS geom, recTime, acId, alt
  FROM pointtable
  WHERE recTime>={} AND recTime<={}
  """.format(t_start, t_end))

spatialdf.createOrReplaceTempView("spatialdf")
spatialdf.show(5, truncate=False)

+--------------------------+----------+-------+-----+
|geom                      |recTime   |acId   |alt  |
+--------------------------+----------+-------+-----+
|POINT (33.63759 -84.43789)|1565186555|DAL1350|10.06|
|POINT (33.63753 -84.43788)|1565186556|DAL1350|10.06|
|POINT (33.63747 -84.43786)|1565186557|DAL1350|10.06|
|POINT (33.63737 -84.43784)|1565186558|DAL1350|10.06|
|POINT (33.63733 -84.43785)|1565186560|DAL1350|10.06|
+--------------------------+----------+-------+-----+
only showing top 5 rows



In [10]:
# Count the number of points after the first query
spatialdf.count()

368875

### Range Rectangular Query around KATL

In [11]:
katl = [33.6366996, -84.4278640, 10.06]  # https://www.airnav.com/airport/katl
r = 0.2 # rectangular query range
vs = 0.3 # vertical threshold unit: x100 feet

In [12]:
# ST_PolygonFromEnvelope (MinX:decimal, MinY:decimal, MaxX:decimal, MaxY:decimal, UUID1, UUID2, ...)
range_query_result = spark.sql(
  """
    SELECT DISTINCT acId
    FROM spatialdf
    WHERE ST_Contains(ST_PolygonFromEnvelope({}, {}, {}, {}), geom) AND alt>{}
  """.format(katl[0]-r, katl[1]-r, katl[0]+r, katl[1]+r, katl[2]+vs))

In [13]:
# Number of flight IDs after the second query
range_query_result.count()

580

In [14]:
range_query_result.show(5)

+-------+
|   acId|
+-------+
|DAL2041|
| UAL241|
|GJS4507|
|DAL1154|
|SKW3742|
+-------+
only showing top 5 rows



In [15]:
df_result = spark.sql(
  """
    SELECT *
    FROM spatialdf
    WHERE ST_Contains(ST_PolygonFromEnvelope({}, {}, {}, {}), geom) AND alt>{}
  """.format(katl[0]-r, katl[1]-r, katl[0]+r, katl[1]+r, katl[2]))

# Count the number of points after the second query
df_result.count()

152508

# Data anonymization

Anonymization is a type of data sanitization technique to remove identifiable information from the data. In this work, we perform two operations to anonymize the data while retaining useful geo-spatial features.

- Normalize the timestamp by the earliest time in the current dataframe.
- Mask the real flight IDs into integers

In [16]:
# create relevant timestamp column
df_result.createOrReplaceTempView("spatialdf")
df = spark.sql(
"""
    SELECT acId, recTime-{} AS t, geom, alt
    FROM spatialdf
""".format(t_start)
)
df.show(5, False)

+-------+---+--------------------------+-----+
|acId   |t  |geom                      |alt  |
+-------+---+--------------------------+-----+
|DAL1350|448|POINT (33.63582 -84.41124)|10.38|
|DAL1350|449|POINT (33.63583 -84.41117)|10.5 |
|DAL1350|450|POINT (33.63576 -84.41111)|10.25|
|DAL1350|451|POINT (33.63577 -84.41103)|10.13|
|DAL1350|665|POINT (33.63472 -84.41763)|10.75|
+-------+---+--------------------------+-----+
only showing top 5 rows



In [17]:
# change acId Into integers
# have to use pyspark ml features
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="acId", outputCol="FacId")
df_new = indexer.fit(df).transform(df).drop('acId')
df_new.show(20, False)

+---+--------------------------+-----+-----+
|t  |geom                      |alt  |FacId|
+---+--------------------------+-----+-----+
|448|POINT (33.63582 -84.41124)|10.38|56.0 |
|449|POINT (33.63583 -84.41117)|10.5 |56.0 |
|450|POINT (33.63576 -84.41111)|10.25|56.0 |
|451|POINT (33.63577 -84.41103)|10.13|56.0 |
|665|POINT (33.63472 -84.41763)|10.75|56.0 |
|666|POINT (33.63471 -84.41828)|10.56|56.0 |
|667|POINT (33.6347 -84.41892) |10.31|56.0 |
|668|POINT (33.6347 -84.41959) |10.38|56.0 |
|669|POINT (33.6347 -84.42027) |10.38|56.0 |
|670|POINT (33.63468 -84.421)  |10.38|56.0 |
|671|POINT (33.63467 -84.42173)|10.38|56.0 |
|672|POINT (33.6347 -84.42251) |10.38|56.0 |
|673|POINT (33.6347 -84.42328) |10.38|56.0 |
|674|POINT (33.63471 -84.42409)|10.38|56.0 |
|675|POINT (33.6347 -84.42492) |10.38|56.0 |
|676|POINT (33.63469 -84.42577)|10.38|56.0 |
|677|POINT (33.6347 -84.42663) |10.38|56.0 |
|678|POINT (33.63472 -84.4275) |10.38|56.0 |
|679|POINT (33.63472 -84.42841)|10.13|56.0 |
|682|POINT

In [18]:
df_new.createOrReplaceTempView("spatialdf")

df = spark.sql(
"""
    SELECT t, CAST(FacId AS Integer), ST_X(geom) as lat, ST_Y(geom) as lon, alt
    FROM spatialdf
"""
)

## Show the data after anonymization

In [19]:
df.show(5, False)

+---+-----+--------+---------+-----+
|t  |FacId|lat     |lon      |alt  |
+---+-----+--------+---------+-----+
|448|56   |33.63582|-84.41124|10.38|
|449|56   |33.63583|-84.41117|10.5 |
|450|56   |33.63576|-84.41111|10.25|
|451|56   |33.63577|-84.41103|10.13|
|665|56   |33.63472|-84.41763|10.75|
+---+-----+--------+---------+-----+
only showing top 5 rows



# Some small modifications to the data

### Modification 1: 
#### Resample the time series with an interval $dt$

In [24]:
dt = 10
t_start = 0
t_end = 3600 * duration

t_interval = list(range(t_start, t_end, dt))
df = df[df.t.isin(t_interval)]

In [25]:
df.show(15)

+---+-----+--------+---------+-----+
|  t|FacId|     lat|      lon|  alt|
+---+-----+--------+---------+-----+
|450|   56|33.63576|-84.41111|10.25|
|670|   56|33.63468|  -84.421|10.38|
|690|   56|33.63457|-84.43889|14.06|
|700|   56| 33.6344|-84.44845|19.13|
|710|   56| 33.6344|-84.45818| 23.5|
|720|   56|33.63442|-84.46875|25.31|
|730|   56|33.63246|   -84.48| 28.5|
|740|   56|33.62812|-84.49124|31.06|
|750|   56| 33.6228|-84.50281|34.63|
|760|   56|33.61735|-84.51479|39.31|
|770|   56|33.61187|-84.52725|44.13|
|780|   56|33.60656|-84.54127|48.63|
|790|   56|33.60111|-84.55442|54.25|
|800|   56|33.59564|-84.56774|59.44|
|810|   56| 33.5874|-84.57992| 64.0|
+---+-----+--------+---------+-----+
only showing top 15 rows



### Modification 2:
Change the origin of the coordinate system to the airport center
- void in real case

In [26]:
df.createOrReplaceTempView("spatialdf")

df2 = spark.sql(
"""
    SELECT t, FacId, lat-{} AS Lat, lon-{} AS Lon, alt
    FROM spatialdf
""".format(katl[0], katl[1])
)

### Save into csv

In [28]:
# Make sure the data types are correct
df.toPandas()
df.dtypes

[('t', 'int'),
 ('FacId', 'int'),
 ('lat', 'double'),
 ('lon', 'double'),
 ('alt', 'double')]

In [29]:
# save data with altitude
csv_name = 'KATL_r_{}_date_{}_range_{}_wAltitude.csv'.format(r, date, duration)
df.toPandas().T.to_csv(csv_name, sep=',', index=False, header=False)
# df.coalesce(1).write.csv(csv_name, sep=',')

# save data without altitude dimension
csv_name = 'KATL_r_{}_date_{}_range_{}.csv'.format(r, date, duration)
df.drop('alt').toPandas().T.to_csv(csv_name, sep=',', index=False, header=False)
# df.coalesce(1).write.csv(csv_name, sep=',')